In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.metrics as sklm
import xgboost as xgb
import matplotlib

C:\Users\hp\Anaconda3\lib\site-packages\distributed\utils.py:134: RuntimeWarning: Couldn't detect a suitable IP address for reaching '8.8.8.8', defaulting to '127.0.0.1': [WinError 10065] A socket operation was attempted to an unreachable host
  % (host, default, e), RuntimeWarning)


In [2]:
train = pd.read_csv('trainreg.csv')
test = pd.read_csv('test.csv')
submissions = pd.read_csv('submissionfile.csv')
vd = pd.read_csv('VariableDefinitions.csv')

In [3]:
from sklearn import preprocessing

def encode_string(data):
    enc = preprocessing.LabelEncoder()
    enc.fit(data)
    enc_features = enc.transform(data)
    ohe = preprocessing.OneHotEncoder()
    encoded = ohe.fit(enc_features.reshape(-1,1))
    return encoded.transform(enc_features.reshape(-1,1)).toarray()

In [4]:
categorical_columns = ['location_type', 'cellphone_access', 'gender_of_respondent', 'relationship_with_head',
                       'marital_status', 'education_level', 'job_type']

Features_enc = encode_string(train['country'])
for col in categorical_columns:
    temp = encode_string(train[col])
    Features_enc = np.concatenate([Features_enc, temp], axis = 1)
    
print(Features_enc.shape)

test_enc = encode_string(test['country'])
for col in categorical_columns:
    temp1 = encode_string(test[col])
    test_enc = np.concatenate([test_enc, temp1], axis = 1)
    
print(test_enc.shape)

(23524, 37)
(10086, 37)


In [5]:
Features_enc = np.concatenate([Features_enc, np.array(train[['household_size', 'age_of_respondent', 'year']])], axis = 1)
print(Features_enc.shape)

test_enc = np.concatenate([test_enc, np.array(test[['household_size', 'age_of_respondent', 'year']])], axis = 1)
print(test_enc.shape)

(23524, 40)
(10086, 40)


In [6]:
scaler = preprocessing.StandardScaler().fit(Features_enc[:, 37:40])
Features_enc[:, 37:40] = scaler.transform(Features_enc[:, 37:40])
print(Features_enc[:5, 35:40])

[[ 0.          1.         -0.35800673 -0.89618796  1.20854126]
 [ 0.          0.          0.53983446  1.88827897  1.20854126]
 [ 0.          1.          0.53983446 -0.77512418  1.20854126]
 [ 0.          0.          0.53983446 -0.29086906  1.20854126]
 [ 0.          0.          1.88659625 -0.77512418  1.20854126]]


In [7]:
scaler1 = preprocessing.StandardScaler().fit(test_enc[:, 37:40])
test_enc[:, 37:40] = scaler1.transform(test_enc[:, 37:40])
print(test_enc[:5, 35:40])

[[ 0.          0.         -0.35198335 -0.51067812  1.20863498]
 [ 0.          0.          1.45583541  0.78010077  1.20863498]
 [ 1.          0.         -0.35198335  2.37820797  1.20863498]
 [ 1.          0.          1.00388072  0.04251284  1.20863498]
 [ 1.          0.         -0.35198335 -1.37119738  1.20863498]]


In [8]:
Labels = np.array(train['bank_account'])
print(Labels)

[ 46.       47.       48.      ... 715.88328 716.88328 717.88328]


In [9]:
import numpy.random as nr
import sklearn.model_selection as ms
## Randomly sample cases to create independent training and test data
nr.seed(9988)
indx = range(Features_enc.shape[0])
indx = ms.train_test_split(indx, test_size = 0.2)
x_train = Features_enc[indx[0],:]
y_train = np.ravel(Labels[indx[0]])
x_test = Features_enc[indx[1],:]
y_test = np.ravel(Labels[indx[1]])

In [10]:
indx1 = range(x_train.shape[0])
indx1 = ms.train_test_split(indx1, test_size = 0.2)
x_train1 = x_train[indx1[0],:]
y_train1 = np.ravel(y_train[indx1[0]])
x_train_val = x_train[indx1[1],:]
y_train_val = np.ravel(y_train[indx1[1]])

In [11]:
print(x_train.shape)
print(x_train1.shape)
print(x_train_val.shape)
print(y_train.shape)
print(y_train1.shape)
print(y_train_val.shape)
print(x_test.shape)
print(y_test.shape)

(18819, 40)
(15055, 40)
(3764, 40)
(18819,)
(15055,)
(3764,)
(4705, 40)
(4705,)


In [21]:
xg_cl1 = xgb.XGBRegressor(objective = 'reg:squarederror', n_estimators = 4000, seed = 123, max_depth = 8,
                           learning_rate=0.01, booster = 'gbtree', base_score = 0.8, subsample = 0.8,
                           reg_lambda = 0.03)

In [13]:
def print_format(f, x, y, z):
    print('Fold %1d     %4.3f         %4.3f        %4.3f'  %  (f, x, y, z))
    
def print_metrics(labels, scores):
    metrics = sklm.precision_recall_fscore_support(labels, scores)
    conf = sklm.confusion_matrix(labels, scores)
    print('                  Confusion matrix')
    print('                 Score positive     Score negative')
    print('True positive    %6d' % conf[0,0] + '            %5d' % conf[0,1])
    print('True negative    %6d' % conf[1,0] + '            %5d' % conf[1,1])
    print('')
    print('Incorrect    %0.2f' % (1 - (sklm.accuracy_score(labels, scores))))
    print('')
    print('              Positive    Negative')
    print('Num case     %0.2f' % metrics[3][0] + '       %0.2f' % metrics[3][1])
    print('Precision       %0.2f' % metrics[0][0] + '       %0.2f' % metrics[0][1])
    print('Recall          %0.2f' % metrics[1][0] + '       %0.2f' % metrics[1][1])
    print('F1              %0.2f' % metrics[2][0] + '       %0.2f' % metrics[2][1])

In [14]:
def print_cv(scores):
    fold = [x + 1 for x in range(len(scores['test_precision_macro']))]
    print('         Precision     Recall,   AUC')
    [print_format(f,x,y,z) for f,x,y,z in zip(fold, scores['test_precision_macro'],
                                             scores['test_recall_macro'],
                                             scores['test_roc_auc'])]
    print('-' * 40)
    print('Mean     %4.3f        %4.3f        %4.3f'  %
         (np.mean(scores['test_precision_macro']), np.mean(scores['test_recall_macro']), np.mean(scores['test_roc_auc'])))
    print('Std      %4.3f        %4.3f      %4.3f'    %
         (np.std(scores['test_precision_macro']), np.std(scores['test_recall_macro']), np.std(scores['test_roc_auc'])))
    
#print_cv(scores)

In [22]:
eval_set = [(x_train1, y_train1), (x_test, y_test)]
xg_cl1.fit(x_train1, y_train1, eval_metric="rmse", eval_set=eval_set, verbose=True, early_stopping_rounds = 200)

[0]	validation_0-rmse:148.67879	validation_1-rmse:147.51349
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 200 rounds.
[1]	validation_0-rmse:148.25717	validation_1-rmse:147.15704
[2]	validation_0-rmse:147.84551	validation_1-rmse:146.80366
[3]	validation_0-rmse:147.43848	validation_1-rmse:146.45883
[4]	validation_0-rmse:147.02646	validation_1-rmse:146.12973
[5]	validation_0-rmse:146.61203	validation_1-rmse:145.79965
[6]	validation_0-rmse:146.21115	validation_1-rmse:145.47218
[7]	validation_0-rmse:145.82864	validation_1-rmse:145.15439
[8]	validation_0-rmse:145.44740	validation_1-rmse:144.83867
[9]	validation_0-rmse:145.06926	validation_1-rmse:144.52852
[10]	validation_0-rmse:144.69208	validation_1-rmse:144.22861
[11]	validation_0-rmse:144.33034	validation_1-rmse:143.93150
[12]	validation_0-rmse:143.97118	validation_1-rmse:143.64159
[13]	validation_0-rmse:143.61951	validation_1-rmse:143.36

[132]	validation_0-rmse:124.17475	validation_1-rmse:130.74829
[133]	validation_0-rmse:124.12415	validation_1-rmse:130.72566
[134]	validation_0-rmse:124.07410	validation_1-rmse:130.70583
[135]	validation_0-rmse:124.00266	validation_1-rmse:130.68968
[136]	validation_0-rmse:123.93625	validation_1-rmse:130.66638
[137]	validation_0-rmse:123.87035	validation_1-rmse:130.64239
[138]	validation_0-rmse:123.81308	validation_1-rmse:130.61742
[139]	validation_0-rmse:123.74838	validation_1-rmse:130.59830
[140]	validation_0-rmse:123.69271	validation_1-rmse:130.57840
[141]	validation_0-rmse:123.62096	validation_1-rmse:130.55887
[142]	validation_0-rmse:123.55810	validation_1-rmse:130.54851
[143]	validation_0-rmse:123.49966	validation_1-rmse:130.53700
[144]	validation_0-rmse:123.43838	validation_1-rmse:130.50876
[145]	validation_0-rmse:123.36746	validation_1-rmse:130.49028
[146]	validation_0-rmse:123.30630	validation_1-rmse:130.47623
[147]	validation_0-rmse:123.24384	validation_1-rmse:130.46840
[148]	va

[265]	validation_0-rmse:118.42910	validation_1-rmse:130.32610
[266]	validation_0-rmse:118.38676	validation_1-rmse:130.32402
[267]	validation_0-rmse:118.34961	validation_1-rmse:130.32915
[268]	validation_0-rmse:118.31120	validation_1-rmse:130.34387
[269]	validation_0-rmse:118.27624	validation_1-rmse:130.35887
[270]	validation_0-rmse:118.24542	validation_1-rmse:130.35898
[271]	validation_0-rmse:118.21109	validation_1-rmse:130.36137
[272]	validation_0-rmse:118.18277	validation_1-rmse:130.36281
[273]	validation_0-rmse:118.14680	validation_1-rmse:130.36523
[274]	validation_0-rmse:118.09891	validation_1-rmse:130.36714
[275]	validation_0-rmse:118.07490	validation_1-rmse:130.36568
[276]	validation_0-rmse:118.04520	validation_1-rmse:130.37291
[277]	validation_0-rmse:118.01111	validation_1-rmse:130.38188
[278]	validation_0-rmse:117.97337	validation_1-rmse:130.38806
[279]	validation_0-rmse:117.92928	validation_1-rmse:130.39305
[280]	validation_0-rmse:117.89546	validation_1-rmse:130.39800
[281]	va

[398]	validation_0-rmse:114.93784	validation_1-rmse:130.87373
[399]	validation_0-rmse:114.91106	validation_1-rmse:130.87880
[400]	validation_0-rmse:114.88242	validation_1-rmse:130.88353
[401]	validation_0-rmse:114.84814	validation_1-rmse:130.88957
[402]	validation_0-rmse:114.83215	validation_1-rmse:130.89278
[403]	validation_0-rmse:114.79459	validation_1-rmse:130.90399
[404]	validation_0-rmse:114.77499	validation_1-rmse:130.91676
[405]	validation_0-rmse:114.75202	validation_1-rmse:130.91586
[406]	validation_0-rmse:114.73273	validation_1-rmse:130.91772
[407]	validation_0-rmse:114.70275	validation_1-rmse:130.91930
[408]	validation_0-rmse:114.67879	validation_1-rmse:130.92424
[409]	validation_0-rmse:114.64810	validation_1-rmse:130.93214
[410]	validation_0-rmse:114.62917	validation_1-rmse:130.93437
Stopping. Best iteration:
[210]	validation_0-rmse:120.41174	validation_1-rmse:130.14511



XGBRegressor(base_score=0.8, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
       importance_type='gain', interaction_constraints=None,
       learning_rate=0.01, max_delta_step=0, max_depth=8,
       min_child_weight=1, missing=nan, monotone_constraints=None,
       n_estimators=4000, n_jobs=0, num_parallel_tree=1,
       objective='reg:squarederror', random_state=123, reg_alpha=0,
       reg_lambda=0.03, scale_pos_weight=1, seed=123, subsample=0.8,
       tree_method=None, validate_parameters=False, verbosity=None)

In [23]:
preds = xg_cl1.predict(x_train_val)

In [24]:
preds[:20]

array([ 55.053402, 107.043755,  49.534046,  58.222282, 186.92996 ,
        53.448486,  49.117546,  52.695587,  35.382282,  57.905758,
        28.57961 ,  53.336266,  51.448765,  34.52467 ,  50.705475,
        57.617046,  53.113407,  54.428986,  60.218494,  52.662758],
      dtype=float32)

In [31]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_absolute_error

mean_absolute_error(y_train_val, preds)

102.13532644948324

In [60]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor

rlf = RandomForestRegressor(n_estimators=1000, max_depth=7, max_features=8, random_state=123,
                            min_samples_split=4)

In [61]:
rlf.fit(x_train1, y_train1)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=7,
           max_features=8, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=4, min_weight_fraction_leaf=0.0,
           n_estimators=1000, n_jobs=1, oob_score=False, random_state=123,
           verbose=0, warm_start=False)

In [62]:
pred_rf = rlf.predict(x_train_val)

In [63]:
pred_rf

array([ 57.35122779,  81.50765702,  73.80204267, ...,  51.89073898,
       174.30844984,  61.67007197])

In [64]:
mean_absolute_error(pred_rf, y_train_val)

100.68923489361251

In [67]:
from sklearn.model_selection import cross_validate
labels_val = y_train1.reshape(y_train1.shape[0],)
scoring = ['precision_macro', 'recall_macro', 'roc_auc']
scores = ms.cross_validate(rlf, x_train1, labels_val, scoring = scoring, cv = 10, return_train_score=True)

ValueError: continuous is not supported

In [68]:
preds_avg = (preds + pred_rf)/2

In [69]:
preds_avg

array([ 56.20231487,  94.2757058 ,  61.66804442, ...,  49.58655525,
       169.12139564,  55.42913694])

In [70]:
mean_absolute_error(preds_avg, y_train_val)

101.25640038404238

In [71]:
from sklearn.model_selection import train_test_split
training, valid, ytraining, yvalid = train_test_split(Features_enc, Labels, test_size=0.5)

In [81]:
from sklearn.linear_model import LinearRegression

model1 = RandomForestRegressor()
model2 = LinearRegression()

In [103]:
rlf.fit(training, ytraining)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=7,
           max_features=8, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=4, min_weight_fraction_leaf=0.0,
           n_estimators=1000, n_jobs=1, oob_score=False, random_state=123,
           verbose=0, warm_start=False)

In [104]:
eval_set = [(x_train1, y_train1), (x_test, y_test)]
xg_cl1.fit(training, ytraining, eval_metric="rmse", eval_set=eval_set, verbose=True, early_stopping_rounds = 200)

preds1 =model1.predict(valid)
preds2 = model2.predict(valid)

test_preds1 = model1.predict(test_enc)
test_preds2 = model2.predict(test_enc)

[0]	validation_0-rmse:148.68552	validation_1-rmse:147.48965
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 200 rounds.
[1]	validation_0-rmse:148.28012	validation_1-rmse:147.08890
[2]	validation_0-rmse:147.88281	validation_1-rmse:146.71262
[3]	validation_0-rmse:147.50600	validation_1-rmse:146.35506
[4]	validation_0-rmse:147.14015	validation_1-rmse:146.00198
[5]	validation_0-rmse:146.76625	validation_1-rmse:145.62518
[6]	validation_0-rmse:146.38118	validation_1-rmse:145.25104
[7]	validation_0-rmse:146.02396	validation_1-rmse:144.90724
[8]	validation_0-rmse:145.65718	validation_1-rmse:144.56421
[9]	validation_0-rmse:145.28105	validation_1-rmse:144.24280
[10]	validation_0-rmse:144.93367	validation_1-rmse:143.91174
[11]	validation_0-rmse:144.59401	validation_1-rmse:143.58850
[12]	validation_0-rmse:144.26379	validation_1-rmse:143.27084
[13]	validation_0-rmse:143.92996	validation_1-rmse:142.95

[132]	validation_0-rmse:127.05788	validation_1-rmse:127.32494
[133]	validation_0-rmse:127.01095	validation_1-rmse:127.29037
[134]	validation_0-rmse:126.96540	validation_1-rmse:127.25598
[135]	validation_0-rmse:126.91741	validation_1-rmse:127.21323
[136]	validation_0-rmse:126.87999	validation_1-rmse:127.17883
[137]	validation_0-rmse:126.84274	validation_1-rmse:127.14265
[138]	validation_0-rmse:126.78841	validation_1-rmse:127.09540
[139]	validation_0-rmse:126.73965	validation_1-rmse:127.04626
[140]	validation_0-rmse:126.68825	validation_1-rmse:126.98482
[141]	validation_0-rmse:126.64837	validation_1-rmse:126.95432
[142]	validation_0-rmse:126.60332	validation_1-rmse:126.91328
[143]	validation_0-rmse:126.56091	validation_1-rmse:126.86596
[144]	validation_0-rmse:126.51718	validation_1-rmse:126.82901
[145]	validation_0-rmse:126.46688	validation_1-rmse:126.79052
[146]	validation_0-rmse:126.42265	validation_1-rmse:126.74977
[147]	validation_0-rmse:126.36937	validation_1-rmse:126.70873
[148]	va

[265]	validation_0-rmse:123.41363	validation_1-rmse:124.42352
[266]	validation_0-rmse:123.39441	validation_1-rmse:124.40723
[267]	validation_0-rmse:123.37414	validation_1-rmse:124.38650
[268]	validation_0-rmse:123.35416	validation_1-rmse:124.36714
[269]	validation_0-rmse:123.34450	validation_1-rmse:124.35818
[270]	validation_0-rmse:123.33838	validation_1-rmse:124.35682
[271]	validation_0-rmse:123.32746	validation_1-rmse:124.34739
[272]	validation_0-rmse:123.30923	validation_1-rmse:124.33147
[273]	validation_0-rmse:123.29848	validation_1-rmse:124.31343
[274]	validation_0-rmse:123.27874	validation_1-rmse:124.30236
[275]	validation_0-rmse:123.26888	validation_1-rmse:124.30197
[276]	validation_0-rmse:123.25773	validation_1-rmse:124.28826
[277]	validation_0-rmse:123.24075	validation_1-rmse:124.27705
[278]	validation_0-rmse:123.22727	validation_1-rmse:124.26701
[279]	validation_0-rmse:123.20183	validation_1-rmse:124.25251
[280]	validation_0-rmse:123.18245	validation_1-rmse:124.24058
[281]	va

[398]	validation_0-rmse:121.69038	validation_1-rmse:123.11878
[399]	validation_0-rmse:121.67847	validation_1-rmse:123.11292
[400]	validation_0-rmse:121.66378	validation_1-rmse:123.10916
[401]	validation_0-rmse:121.65466	validation_1-rmse:123.10329
[402]	validation_0-rmse:121.63913	validation_1-rmse:123.08639
[403]	validation_0-rmse:121.62787	validation_1-rmse:123.07453
[404]	validation_0-rmse:121.61494	validation_1-rmse:123.06644
[405]	validation_0-rmse:121.60824	validation_1-rmse:123.06001
[406]	validation_0-rmse:121.59517	validation_1-rmse:123.04775
[407]	validation_0-rmse:121.58219	validation_1-rmse:123.03416
[408]	validation_0-rmse:121.57011	validation_1-rmse:123.02266
[409]	validation_0-rmse:121.55843	validation_1-rmse:123.01322
[410]	validation_0-rmse:121.55095	validation_1-rmse:123.01325
[411]	validation_0-rmse:121.54016	validation_1-rmse:123.00803
[412]	validation_0-rmse:121.52839	validation_1-rmse:123.00131
[413]	validation_0-rmse:121.51812	validation_1-rmse:122.99438
[414]	va

[531]	validation_0-rmse:120.21119	validation_1-rmse:122.08709
[532]	validation_0-rmse:120.19924	validation_1-rmse:122.08693
[533]	validation_0-rmse:120.18265	validation_1-rmse:122.07307
[534]	validation_0-rmse:120.17487	validation_1-rmse:122.06290
[535]	validation_0-rmse:120.16779	validation_1-rmse:122.06188
[536]	validation_0-rmse:120.15327	validation_1-rmse:122.05853
[537]	validation_0-rmse:120.14407	validation_1-rmse:122.05681
[538]	validation_0-rmse:120.12828	validation_1-rmse:122.04276
[539]	validation_0-rmse:120.11507	validation_1-rmse:122.03414
[540]	validation_0-rmse:120.09592	validation_1-rmse:122.02439
[541]	validation_0-rmse:120.08939	validation_1-rmse:122.02248
[542]	validation_0-rmse:120.07986	validation_1-rmse:122.01726
[543]	validation_0-rmse:120.06597	validation_1-rmse:122.00903
[544]	validation_0-rmse:120.05424	validation_1-rmse:122.00025
[545]	validation_0-rmse:120.04230	validation_1-rmse:121.99352
[546]	validation_0-rmse:120.02704	validation_1-rmse:121.98273
[547]	va

...

[3027]	validation_0-rmse:111.43143	validation_1-rmse:116.09050
[3028]	validation_0-rmse:111.43381	validation_1-rmse:116.08485
[3029]	validation_0-rmse:111.43475	validation_1-rmse:116.08433
[3030]	validation_0-rmse:111.43315	validation_1-rmse:116.08614
[3031]	validation_0-rmse:111.43449	validation_1-rmse:116.08868
[3032]	validation_0-rmse:111.43478	validation_1-rmse:116.08556
[3033]	validation_0-rmse:111.43497	validation_1-rmse:116.08519
[3034]	validation_0-rmse:111.43228	validation_1-rmse:116.08611
[3035]	validation_0-rmse:111.43156	validation_1-rmse:116.08875
[3036]	validation_0-rmse:111.42955	validation_1-rmse:116.09422
[3037]	validation_0-rmse:111.42894	validation_1-rmse:116.09264
[3038]	validation_0-rmse:111.42834	validation_1-rmse:116.09283
[3039]	validation_0-rmse:111.42764	validation_1-rmse:116.09320
[3040]	validation_0-rmse:111.42803	validation_1-rmse:116.09106
[3041]	validation_0-rmse:111.42773	validation_1-rmse:116.09119
[3042]	validation_0-rmse:111.42790	validation_1-rmse:11

[3158]	validation_0-rmse:111.37276	validation_1-rmse:116.06915
[3159]	validation_0-rmse:111.37186	validation_1-rmse:116.07110
[3160]	validation_0-rmse:111.37154	validation_1-rmse:116.07471
[3161]	validation_0-rmse:111.37001	validation_1-rmse:116.07657
[3162]	validation_0-rmse:111.37090	validation_1-rmse:116.07764
[3163]	validation_0-rmse:111.36935	validation_1-rmse:116.07604
[3164]	validation_0-rmse:111.36809	validation_1-rmse:116.08104
[3165]	validation_0-rmse:111.36928	validation_1-rmse:116.07652
[3166]	validation_0-rmse:111.37058	validation_1-rmse:116.07778
[3167]	validation_0-rmse:111.36986	validation_1-rmse:116.07705
[3168]	validation_0-rmse:111.36938	validation_1-rmse:116.07730
[3169]	validation_0-rmse:111.36776	validation_1-rmse:116.07623
[3170]	validation_0-rmse:111.36876	validation_1-rmse:116.07328
[3171]	validation_0-rmse:111.36900	validation_1-rmse:116.07026
[3172]	validation_0-rmse:111.36939	validation_1-rmse:116.06882
[3173]	validation_0-rmse:111.36955	validation_1-rmse:11

[3289]	validation_0-rmse:111.34689	validation_1-rmse:116.03361
[3290]	validation_0-rmse:111.34544	validation_1-rmse:116.03237
[3291]	validation_0-rmse:111.34591	validation_1-rmse:116.03172
[3292]	validation_0-rmse:111.34676	validation_1-rmse:116.03279
[3293]	validation_0-rmse:111.34491	validation_1-rmse:116.03298
[3294]	validation_0-rmse:111.34441	validation_1-rmse:116.03146
[3295]	validation_0-rmse:111.34352	validation_1-rmse:116.03098
[3296]	validation_0-rmse:111.34323	validation_1-rmse:116.02888
[3297]	validation_0-rmse:111.34069	validation_1-rmse:116.02708
[3298]	validation_0-rmse:111.33991	validation_1-rmse:116.02799
[3299]	validation_0-rmse:111.34001	validation_1-rmse:116.02688
[3300]	validation_0-rmse:111.34042	validation_1-rmse:116.02851
[3301]	validation_0-rmse:111.33843	validation_1-rmse:116.02928
[3302]	validation_0-rmse:111.33688	validation_1-rmse:116.02565
[3303]	validation_0-rmse:111.33590	validation_1-rmse:116.02336
[3304]	validation_0-rmse:111.33297	validation_1-rmse:11

[3420]	validation_0-rmse:111.30455	validation_1-rmse:116.03619
[3421]	validation_0-rmse:111.30297	validation_1-rmse:116.03865
[3422]	validation_0-rmse:111.30288	validation_1-rmse:116.04120
[3423]	validation_0-rmse:111.30320	validation_1-rmse:116.04366
[3424]	validation_0-rmse:111.30283	validation_1-rmse:116.04412
[3425]	validation_0-rmse:111.30241	validation_1-rmse:116.04429
[3426]	validation_0-rmse:111.30338	validation_1-rmse:116.04636
[3427]	validation_0-rmse:111.30344	validation_1-rmse:116.04830
[3428]	validation_0-rmse:111.30445	validation_1-rmse:116.04882
[3429]	validation_0-rmse:111.30730	validation_1-rmse:116.04954
[3430]	validation_0-rmse:111.30775	validation_1-rmse:116.05477
[3431]	validation_0-rmse:111.30562	validation_1-rmse:116.05831
[3432]	validation_0-rmse:111.30464	validation_1-rmse:116.05590
[3433]	validation_0-rmse:111.30392	validation_1-rmse:116.05563
[3434]	validation_0-rmse:111.30453	validation_1-rmse:116.05530
[3435]	validation_0-rmse:111.30512	validation_1-rmse:11

[3551]	validation_0-rmse:111.29925	validation_1-rmse:116.10698
Stopping. Best iteration:
[3351]	validation_0-rmse:111.30898	validation_1-rmse:116.01502



In [105]:
preds1

array([ 95.56538065,  48.57168965, 111.38746545, ..., 228.82615996,
        30.59635893, 117.73472749])

In [106]:
preds2

array([ 65.125 ,  54.6875,  55.    , ..., 175.875 ,  61.5   ,  52.875 ])

In [107]:
stacked_predictions = np.column_stack((preds1, preds2))
stacked_test_predictions = np.column_stack((test_preds1, test_preds2))

In [108]:
stacked_test_predictions.shape

(10086, 2)

In [109]:
meta_model = LinearRegression()

meta_model.fit(stacked_predictions, yvalid)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [110]:
final_predictions = meta_model.predict(stacked_test_predictions)

In [101]:
final_predictions.shape

(10086,)

In [111]:
from sklearn.model_selection import RandomizedSearchCV

In [116]:
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 100, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

In [117]:
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [118]:
rf = RandomForestRegressor()

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid,
                                n_iter = 100, cv = 5, verbose=2, random_state=40, n_jobs = -1)

In [ ]:
rf_random.fit(x_train1, y_train1)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 39.3min
